In [1]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import hist, processor
from pocket_coffea.parameters import defaults
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
from hist import Hist

import sys
import os

from custom_cut_functions import *
from custom_cuts import *

/t3home/mmalucch/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/util.py:154: FutureWarning: In coffea version v2023.3.0 (target date: 31 Mar 2023), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [2]:
filename ="root://storage01.lcg.cscs.ch:1096//pnfs/lcg.cscs.ch/cms/trivcat//store/mc/Run3Summer22EENanoAODv12/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/Poisson60KeepRAW_130X_mcRun3_2022_realistic_postEE_v6-v2/50000/fc603037-ef65-4bbf-9cef-934ecec40bbe.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_stop=3).events()
print("Events read:", len(events))

Events read: 3


In [19]:
# print(events.fields)
# print(events.Electron.fields)
print(events.GenPart)
sum_p4=events.GenPart[:,2]+events.GenPart[:,3]
print(sum_p4.mass)
print(sum_p4.pt)
print(events.GenPart[:,2].pt)
print(events.GenPart[:,3].pt)

[[GenParticle, GenParticle, GenParticle, ... GenParticle, GenParticle, GenParticle]]
[535, 585, 338]
[51.2, 29, 84.4]
[257, 78.2, 159]
[217, 98, 75.5]


In [4]:
default_parameters = defaults.get_default_parameters()
defaults.register_configuration_dir("config_dir", "/params")

parameters = defaults.merge_parameters_from_files(    default_parameters,
    f"params/object_preselection.yaml",
    f"params/triggers.yaml",
    # f"{localdir}/params/lepton_scale_factors.yaml",
    # f"{localdir}/params/plotting_style.yaml",
    update=True,
)
# print(parameters["object_preselection"]["Muon"]["pt"])

In [5]:
print(events.LHEPart.pdgId)

# Select b-quarks at LHE level
isOutgoing = events.LHEPart.status == 1
print(isOutgoing)

isB = abs(events.LHEPart.pdgId) == 5
# print only events with at least one b-quark
for     i in range(len(isB)):
    if len(isB[isB][i]) > 0:
        print(i)
        print(events.LHEPart.pdgId[i])
        break
bquarks = events.LHEPart[isB & isOutgoing]

[[21, 21, 25, 25, 21], [21, 21, 25, 25, ... 21, 25, 25, 21], [21, 21, 25, 25, 21]]
[[False, False, True, True, True], [False, ... [False, False, True, True, True]]
20
[5, 21, 25, 25, 5]


In [6]:
e=events[20]
print(e.LHEPart.pdgId)
pdg_gen = e.GenPart.pdgId
print(pdg_gen==5)
# select b-quarks at Gen level
isB = abs(pdg_gen) == 5
# select higgs at Gen level
isH = pdg_gen == 25
print(pdg_gen[isB | isH])

b_higgs = e.GenPart[isB | isH]


[5, 21, 25, 25, 5]
[True, False, False, False, True, False, ... False, False, False, False, False]
[5, 25, 25, 5, 25, 25, 25, 25, 25, 25, 25, ... 5, -5, 5, -5, 5, 5, -5, -5, 5, -5, 5]


In [7]:
isHiggs = b_higgs.pdgId == 25
isHard = b_higgs.hasFlags(["fromHardProcess"])
hasTwoChildren = ak.num(b_higgs.childrenIdxG, axis=1) == 2
# print(events.GenPart.childrenIdxG)
# print(ak.num(events.GenPart.childrenIdxG, axis=2))

print(len(e.GenPart.pdgId), e.GenPart.pdgId)
print(len(e.GenPart.childrenIdxG), e.GenPart.childrenIdxG)
print(len(e.GenPart.children.pdgId), e.GenPart.children.pdgId
      )

print(len(b_higgs), b_higgs.pdgId
      )
print(len(b_higgs.childrenIdxG), b_higgs.childrenIdxG
      )
print(len(b_higgs.children.pdgId), b_higgs.children.pdgId
      )

# print(isHiggs)
# print(isHard)
# print(hasTwoChildren)

102 [5, 21, 25, 25, 5, 25, 25, 25, 25, 25, ... -11, 221, 111, 111, 111, 22, 22, 22, 22]
102 [[1282, 1283, 1284], [], [1285], [1286], [1298, ... [1380], [1381], [], [], [], []]
102 [[25, 25, 5], [], [25], [25], [5], [25], ... 22], [22], [22], [22], [], [], [], []]
31 [5, 25, 25, 5, 25, 25, 25, 25, 25, 25, 25, ... 5, -5, 5, -5, 5, 5, -5, -5, 5, -5, 5]
31 [[1282, 1283, 1284], [1285], [1286], [1298], ... [1323], [], [], [], [], [], []]
31 [[25, 25, 5], [25], [25], [5], [25], [25, ... 111, 111], [5], [], [], [], [], [], []]


In [28]:
i = 1

j = i + 1
gen = events.GenPart#[i:j]
# print("gen", gen.pdgId)
isHiggs = gen.pdgId == 25
isHard = gen.hasFlags(["fromHardProcess"])
isLastCopy = gen.hasFlags(["isLastCopy"])
isFirstCopy = gen.hasFlags(["isFirstCopy"])
isB = abs(gen.pdgId) == 5
hasTwoChildren = ak.num(gen.childrenIdxG, axis=2)==2
print(hasTwoChildren)
print(gen.childrenIdxG)
print(ak.num(gen.childrenIdxG, axis=2))



# print(hasTwoChildren[isHiggs & isLastCopy])
# print(gen[isHiggs & isHard].childrenIdxG)

# print(gen[isHiggs].hasFlags(["isLastCopy"]))
# print(gen[isHiggs].children.pdgId)
# print(gen[isHiggs & hasTwoChildren].children.pdgId)
# print(gen[isHiggs].childrenIdxG)

higgs = gen[isHiggs & isHard & isLastCopy & hasTwoChildren]
# higgs_loose = gen[isHiggs & isHard & isLastCopy]
# num_higgs = ak.num(higgs_loose, axis=1)
# print(num_higgs, len(num_higgs))
# print(gen[num_higgs!=2].pdgId, len(gen[num_higgs==2].pdgId))
print(higgs.childrenIdxG, higgs.children.pdgId)
bquarks = ak.flatten(higgs.children, axis=2)  # [isB & isHard & isFirstCopy]
tot_b = gen[isB & isHard]
b_unflattened = higgs.children
print(b_unflattened.pdgId)
print(b_unflattened.genPartIdxMother)
print(bquarks.pdgId)
print(bquarks.genPartIdxMother)

all_higgs = gen[isHiggs & isHard]
print("pt", higgs.pt, len(higgs.pt))


b_pairs = ak.combinations(bquarks, 2)
print(b_pairs["0"].pdgId)
print(b_pairs["1"].pdgId)
print(b_pairs["0"].genPartIdxMother)
print(b_pairs["1"].genPartIdxMother)


# given each bquark pair, i want to check if the two bquarks come from the same higgs or not
same_higgs = (b_pairs["0"].genPartIdxMother == b_pairs["1"].genPartIdxMother)

print(same_higgs)
b_pairs = b_pairs[same_higgs]
print(b_pairs)

b_pairs_idx = ak.argcombinations(bquarks, 2)
print(b_pairs_idx )

unz= ak.unzip(b_pairs_idx)
print(unz)

b_pairs_idx=b_pairs_idx[same_higgs]
print(b_pairs_idx)
print(ak.unzip(b_pairs_idx))
b_pair_first =b_pairs_idx[:,0]
b_pair_second =b_pairs_idx[:,1]

print(b_pair_first)
print(b_pair_second)


print(b_pair_first[:]["0"])

b11=bquarks[:,b_pair_first[:]["0"]]
print(b11.pdgId, b11.genPartIdxMother, b11)
b12=bquarks[:,b_pair_first[:]["1"]]
print(b12.pdgId, b12.genPartIdxMother, b12)
b21=bquarks[:,b_pair_second[:]["0"]]
print(b21.pdgId, b21.genPartIdxMother, b21)
b22=bquarks[:,b_pair_second[:]["1"]]
print(b22.pdgId, b22.genPartIdxMother, b22)



provenance = ak.zeros_like(bquarks.pdgId)
provenance = ak.to_numpy(provenance)


for k in [0,1]:
    print(k)
    print(b_pairs_idx[:,k]["0"], b_pairs_idx[:,k]["1"])
    provenance[:,b_pairs_idx[:,k]["0"]] = k+1
    provenance[:,b_pairs_idx[:,k]["1"]] = k+1

print(provenance)




# add a field to bquarks which says if the bquark comes from the first or second higgs
bquarks=ak.with_field(bquarks, provenance, "provenance")
print(bquarks.provenance)
print(bquarks.pdgId)
print(bquarks.genPartIdxMother)

[[False, False, False, False, False, False, ... False, False, False, False, False]]
[[[2, 3, 4], [], [5], [6], [11, 12, 22, 23, ... [], [], [], [200], [201], [], []]]
[[3, 0, 1, 1, 11, 1, 1, 1, 1, 1, 1, 1, 1, ... 0, 0, 1, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0]]
[[[17, 18], [19, 20]], [[87, 88], [89, 90]], [[154, 155], [156, 157]]] [[[5, -5], [5, -5]], [[5, -5], [5, -5]], [[5, -5], [5, -5]]]
[[[5, -5], [5, -5]], [[5, -5], [5, -5]], [[5, -5], [5, -5]]]
[[[15, 15], [16, 16]], [[14, 14], [15, 15]], [[15, 15], [16, 16]]]
[[5, -5, 5, -5], [5, -5, 5, -5], [5, -5, 5, -5]]
[[15, 15, 16, 16], [14, 14, 15, 15], [15, 15, 16, 16]]
pt [[265, 209], [87.2, 108], [162, 72.8]] 3
[[5, 5, 5, -5, -5, 5], [5, 5, 5, -5, -5, 5], [5, 5, 5, -5, -5, 5]]
[[-5, 5, -5, 5, -5, -5], [-5, 5, -5, 5, -5, -5], [-5, 5, -5, 5, -5, -5]]
[[15, 15, 15, 15, 15, 16], [14, 14, 14, 14, 14, 15], [15, 15, 15, 15, 15, 16]]
[[15, 16, 16, 16, 16, 16], [14, 15, 15, 15, 15, 15], [15, 16, 16, 16, 16, 16]]
[[True, False, False, False, False, True,

In [27]:
# compute invariant mass of each bquark pair with the same provenance
higgs1=b11+b12
print(b11.mass)
print(b12.mass)
print(b11.px)
print(b12.px)
print(b11.py)
print(b12.py)
print(b11.pz)
print(b12.pz)
print(b11.energy)
print(b12.energy)
print(higgs1)
print(higgs1.mass)


[[0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[73.1, 73.1, 73.1], [-41.6, -41.6, -41.6], [-46.4, -46.4, -46.4]]
[[-43.7, -43.7, -43.7], [59.8, 59.8, 59.8], [-107, -107, -107]]
[[-105, -105, -105], [-14.2, -14.2, -14.2], [-18.3, -18.3, -18.3]]
[[-159, -159, -159], [-71.3, -71.3, -71.3], [73.4, 73.4, 73.4]]
[[111, 111, 111], [-29.4, -29.4, -29.4], [201, 201, 201]]
[[171, 171, 171], [-223, -223, -223], [139, 139, 139]]
[[170, 170, 170], [52.9, 52.9, 52.9], [207, 207, 207]]
[[237, 237, 237], [242, 242, 242], [190, 190, 190]]
[[{x: 29.4, y: -264, z: 282, t: 407}, {x: 29.4, ... y: 55.1, z: 340, t: 397}]]
[[125, 125, 125], [124, 124, 124], [125, 125, 125]]


In [107]:

minDR=0.4
# Compute deltaR(b, jet) and save the nearest jet (deltaR matching)
deltaR = ak.flatten(bquarks.metric_table(events.Jet), axis=2)
# keeping only the pairs with a deltaR min
maskDR = deltaR<  minDR
deltaRcut = deltaR[maskDR]
idx_pairs_sorted = ak.argsort(deltaRcut, axis=1)
pairs = ak.argcartesian([bquarks, events.Jet])[maskDR]
pairs_sorted = pairs[idx_pairs_sorted]
idx_bquarks, idx_JetGood = ak.unzip(pairs_sorted)